In [5]:
import os
import cv2
import numpy as np
from scipy.io import loadmat

# Define directories
data_dir = './data/MPIIFaceGaze'
yolo_dir = './data/yolo'
train_participants = [f'p{i:02d}' for i in range(12)]  # p00 to p11
val_participants = [f'p{i:02d}' for i in range(12, 15)]  # p12 to p14

# Create directories
os.makedirs(os.path.join(yolo_dir, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(yolo_dir, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(yolo_dir, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(yolo_dir, 'labels', 'val'), exist_ok=True)

def compute_bbox(landmarks, margin=0.2):
    """Compute bounding box from landmarks with margin."""
    x_min, y_min = np.min(landmarks, axis=0)
    x_max, y_max = np.max(landmarks, axis=0)
    w, h = x_max - x_min, y_max - y_min
    x_min = max(0, x_min - w * margin)
    x_max = min(448, x_max + w * margin)
    y_min = max(0, y_min - h * margin)
    y_max = min(448, y_max + h * margin)
    return [x_min, y_min, x_max, y_max]

def bbox_to_yolo(bbox, img_width=448, img_height=448):
    """Convert bounding box to YOLO format (normalized center_x, center_y, width, height)."""
    x_min, y_min, x_max, y_max = bbox
    center_x = (x_min + x_max) / 2 / img_width
    center_y = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return center_x, center_y, width, height

for participants, split in [(train_participants, 'train'), (val_participants, 'val')]:
    for pid in participants:
        annotation_file = os.path.join(data_dir, pid, f'{pid}.txt')
        with open(annotation_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                img_rel_path = parts[0]  # e.g., day01/001.mat
                img_path = os.path.join(data_dir, pid, img_rel_path)
                # Load image from .mat
                try:
                    data = loadmat(img_path)
                    image = data['image']  # Adjust key if necessary
                except KeyError:
                    print(f"Key 'image' not found in {img_path}. Check .mat file structure.")
                    continue
                # Save image as JPEG
                img_name = f"{pid}_{img_rel_path.replace('/', '_').replace('.mat', '.jpg')}"
                img_save_path = os.path.join(yolo_dir, 'images', split, img_name)
                cv2.imwrite(img_save_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
                
                # Extract landmarks
                landmarks = np.array([float(x) for x in parts[3:15]]).reshape(6, 2)
                # Compute bounding boxes
                face_bbox = compute_bbox(landmarks)
                left_eye_bbox = compute_bbox(landmarks[0:2])  # Left eye corners
                right_eye_bbox = compute_bbox(landmarks[2:4])  # Right eye corners
                
                # Convert to YOLO format
                face_yolo = bbox_to_yolo(face_bbox)
                left_eye_yolo = bbox_to_yolo(left_eye_bbox)
                right_eye_yolo = bbox_to_yolo(right_eye_bbox)
                
                # Write to label file
                label_name = img_name.replace('.jpg', '.txt')
                label_path = os.path.join(yolo_dir, 'labels', split, label_name)
                with open(label_path, 'w') as lf:
                    lf.write(f"0 {face_yolo[0]:.6f} {face_yolo[1]:.6f} {face_yolo[2]:.6f} {face_yolo[3]:.6f}\n")
                    lf.write(f"1 {left_eye_yolo[0]:.6f} {left_eye_yolo[1]:.6f} {left_eye_yolo[2]:.6f} {left_eye_yolo[3]:.6f}\n")
                    lf.write(f"2 {right_eye_yolo[0]:.6f} {right_eye_yolo[1]:.6f} {right_eye_yolo[2]:.6f} {right_eye_yolo[3]:.6f}\n")

ValueError: Unknown mat file type, version 50, 50

In [ ]:
# Add to script for debugging
image_with_boxes = image.copy()
for bbox in [face_bbox, left_eye_bbox, right_eye_bbox]:
    x1, y1, x2, y2 = [int(x) for x in bbox]
    cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv2.imwrite(f"debug_{img_name}", cv2.cvtColor(image_with_boxes, cv2.COLOR_RGB2BGR))